<h1>Checking the low recall when aligning M1 data</h1>

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [4]:
from discretisation.preprocessing import FileLoader
from models import HyperPars as AlignmentHyperPars
from discretisation.adduct_cluster import AdductCluster, Peak
from shared_bin_matching import SharedBinMatching as Aligner
from ground_truth import GroundTruth

Define input parameters

In [4]:
input_dir = '/home/joewandy/git/metabolomics_tools/alignment/input/M1_4'
database_file = None
transformation_file = '/home/joewandy/git/metabolomics_tools/discretisation/mulsubs/pos_transformations.yml'

In [5]:
hp = AlignmentHyperPars()    
hp.within_file_mass_tol = 10
hp.within_file_rt_tol = 5
hp.across_file_mass_tol = 30
hp.across_file_rt_tol = 100
hp.alpha_mass = 1.0
hp.dp_alpha = 100.0
hp.t = 0
hp.mass_clustering_n_iterations = 100
hp.rt_clustering_nsamps = 200
hp.rt_clustering_burnin = 100

print hp

Hyperparameters across_file_mass_tol=30, across_file_rt_tol=100, alpha_mass=1.0, beta=0.1, dp_alpha=100.0, mass_clustering_n_iterations=100, rt_clustering_burnin=100, rt_clustering_nsamps=200, t=0, within_file_mass_tol=10, within_file_rt_tol=5


In [6]:
loader = FileLoader()
data_list = loader.load_model_input(input_dir, database_file, 0, 0, make_bins=False)

5842 features read from M1_1.txt
7515 features read from M1_2.txt
9132 features read from M1_3.txt
5876 features read from M1_4.txt


For some reasons, the cell below that does precursor clustering for each file takes **a lot** longer to run in the notebook vs. when run outside ... Not sure why??!

In [ ]:
clustering_results = []
for peak_data in data_list:

    ac = AdductCluster(mass_tol=hp.within_file_mass_tol, rt_tol=hp.within_file_rt_tol, 
                       alpha=hp.alpha_mass, mh_biggest=True, transformation_file=transformation_file, verbose=2)

    peak_list = peak_data.features
    ac.init_from_list(peak_list)

    ac.init_vb()
    for n in range(hp.mass_clustering_n_iterations):
        print "VB step %d file %d " % (n, j)
        sys.stdout.flush()
        ac.vb_step()
        
    clustering_results.append(ac)

<hr/>

Workaround by loading results produced from outside the notebook

In [5]:
aligner = Aligner.resume_from('/home/joewandy/git/metabolomics_tools/alignment/input/M1_4/results.project')

Project loaded from /home/joewandy/git/metabolomics_tools/alignment/input/M1_4/results.project time taken = 18.5390169621


In [6]:
data_list = aligner.data_list
hp = aligner.hp
file_adduct_clusterers = aligner.clustering_results # list of adduct clusterer for each file
file_clusterings = aligner.file_data # dict of file idx to the list of clusters in that file

Find some big clusters in the first file. We have performed MAP assignment of each peak feature into its most likely cluster.

In [7]:
def plot_biggest(file_idx, threshold):

    ac = file_adduct_clusterers[file_idx]
    clusters_list = file_clusterings[file_idx]
    singleton_count = 0
    
    big_clusters = []
    biggest = clusters_list[0]
    for cluster in clusters_list:
        if cluster.N == 1:
            singleton_count += 1
        if cluster.N >= threshold:
            big_clusters.append(cluster)
            if cluster.N >= biggest.N:
                biggest = cluster

    print "Singleton count {}".format(singleton_count)
    print "{} big clusters found".format(len(big_clusters))
    print "Biggest has {} members".format(biggest.N)

    for c in big_clusters:
        ac.cluster_plot(c)

In [ ]:
plot_biggest(file_idx=0, threshold=4)

In [ ]:
plot_biggest(file_idx=1, threshold=4)

Print out all the aligned peaksets

In [11]:
aligned_peaksets = []
i = 0
for i in range(len(aligner.alignment_results)):
    peakset = aligner.alignment_results[i].peakset
    aligned_peaksets.append(peakset)

Load the ground truth and check the annotations

In [12]:
file_list = aligner.file_list
gt_file = '/home/joewandy/git/metabolomics_tools/alignment/input/M1_4/ground_truth/ground_truth.txt'
gt = GroundTruth(gt_file, file_list, data_list)

Loaded 1007 ground truth entries


In [13]:
def found_in(gt_entry, aligned_peaksets):
    for ps in aligned_peaksets:
        ps_keys = [f._get_key() for f in ps]
        for f in gt_entry:
            if f._get_key() not in ps_keys:
                all_found = False
        if all_found:
            return True
    return False

In [21]:
groups = gt.gt_features
not_found_list = []
found_list = []
for group in groups:
    found = found_in(group, aligned_peaksets)
    if not found: # store the not-found ground truth entries
        not_found_list.append(group)
    else:
        found_list.append(group)

In [22]:
print "Aligned peaksets that agree with ground truth = %d/%d" % (len(found_list), len(groups)) 
print "Aligned peaksets that disagree with ground truth = %d/%d" % (len(not_found_list), len(groups)) 

Aligned peaksets that agree with ground truth = 691/1007
Aligned peaksets that disagree with ground truth = 316/1007


Print the found ones

In [56]:
i = 0
for group in found_list:
    print "Group %d" % i
    i += 1
    for f in group:
        key = f._get_key()
        annot = aligner.annotations[key]
        print "- id %s mass %.4f rt %.2f MAP_trans %s" % ((key, f.mass, f.rt, annot))

Group 0
- id (117, 0) mass 128.9968 rt 223.98 MAP_trans M+H@127.98956(0.7973)
- id (148, 1) mass 128.9958 rt 226.02 MAP_trans M+H@127.98884(1.0000)
- id (229, 2) mass 128.9965 rt 223.98 MAP_trans M+H@127.98918(1.0000)
- id (197, 3) mass 128.9964 rt 220.02 MAP_trans M+H@127.98913(0.9963)
Group 1
- id (142, 0) mass 116.0774 rt 222.00 MAP_trans M+H@115.07012(1.0000)
- id (156, 1) mass 116.0773 rt 222.00 MAP_trans M+H@115.07009(1.0000)
- id (8574, 2) mass 116.0777 rt 222.00 MAP_trans M+H@115.07039(1.0000)
- id (88, 3) mass 116.0782 rt 220.02 MAP_trans M+H@115.07096(1.0000)
Group 2
- id (287, 0) mass 279.0475 rt 222.00 MAP_trans M+H@278.04028(1.0000)
- id (1302, 1) mass 279.0507 rt 222.00 MAP_trans M+H@278.04348(1.0000)
- id (1413, 2) mass 279.0473 rt 222.00 MAP_trans M+H@278.04023(1.0000)
- id (300, 3) mass 279.0507 rt 220.02 MAP_trans M+H@278.04302(1.0000)
Group 3
- id (303, 0) mass 200.0499 rt 223.98 MAP_trans M+H@199.04277(1.0000)
- id (257, 1) mass 200.0500 rt 226.02 MAP_trans M+H@199.

Print the not-found ones

In [57]:
def find_overlap(gt_entry, aligned_peaksets):
    overlap = []
    for ps in aligned_peaksets:
        ps_keys = [f._get_key() for f in ps]
        any_found = False
        for f in gt_entry:
            if f._get_key() in ps_keys:
                any_found = True
        if any_found:
            overlap.append(ps)
    return overlap

def print_peakset(peakset):
    print "\tPeakset"
    for f in peakset:
        key = f._get_key()
        annot = aligner.annotations[key]
        print "\t- id %s mass %.4f rt %.2f MAP_trans %s" % ((key, f.mass, f.rt, annot))    

In [58]:
for group in not_found_list:
    
    print "Ground Truth Group %d" % i
    i += 1
    for f in group:
        key = f._get_key()
        print "- id %s mass %.4f rt %.2f" % ((key, f.mass, f.rt))
    
    print "Overlapping peaksets:"
    overlap = find_overlap(group, aligned_peaksets)
    for ps in overlap:
        print_peakset(ps)
    print

Ground Truth Group 691
- id (157, 0) mass 133.0316 rt 226.02
- id (241, 1) mass 133.0371 rt 226.02
- id (312, 2) mass 133.0342 rt 228.00
- id (125, 3) mass 133.0340 rt 222.00
Overlapping peaksets:
	Peakset
	- id (157, 0) mass 133.0316 rt 226.02 MAP_trans [M-CO2]+Na [C13]@153.02872(0.7968)
	- id (125, 3) mass 133.0340 rt 222.00 MAP_trans [M-CO2]+Na [C13]@153.03164(0.7972)

Ground Truth Group 692
- id (215, 0) mass 156.0391 rt 226.02
- id (279, 1) mass 156.0393 rt 226.02
- id (324, 2) mass 156.0412 rt 226.02
- id (171, 3) mass 156.0426 rt 222.00
Overlapping peaksets:
	Peakset
	- id (215, 0) mass 156.0391 rt 226.02 MAP_trans M+H@155.03176(1.0000)
	- id (324, 2) mass 156.0412 rt 226.02 MAP_trans M+H@155.03399(0.8923)
	- id (171, 3) mass 156.0426 rt 222.00 MAP_trans M+H@155.03534(1.0000)
	Peakset
	- id (279, 1) mass 156.0393 rt 226.02 MAP_trans [M-CO]+H [C13]@182.02356(1.0000)

Ground Truth Group 693
- id (221, 0) mass 198.0324 rt 223.98
- id (318, 1) mass 198.0328 rt 223.98
- id (719, 2) m

Some peaks seem to have disappeared from the output aligned peaksets? This looks like a bug, which explains the lower recall ... For example, peak (1327, 1) below ..

In [53]:
print "Found in input file?"
for f in data_list[1].features:
    key = f._get_key()
    if (1327, 1) == key:
        print "- id %s mass %.4f rt %.2f" % ((key, f.mass, f.rt))

# check in the output of first-stage clustering
print "\nAnd also in the clustering"
first_file_clusterings = file_clusterings[1]
for cluster in first_file_clusterings:
    member_keys = [f._get_key() for f, poss in cluster.members]
    if (1327, 1) in member_keys:
        print "Cluster %d %.4f %.2f" % (cluster.id, cluster.mu_mass, cluster.mu_rt)
        print member_keys   
        for f, poss in cluster.members:
            print "- id %s mass %.4f rt %.2f" % ((f._get_key(), f.mass, f.rt))

# check in the output aligned peaksets
print "\nBut missing in the output ??!!"
for ps in aligned_peaksets:
    ps_keys = [f._get_key() for f in ps]
    if (1327, 1) in ps_keys:
        print ps_keys

Found in input file?
- id (1327, 1) mass 284.0365 rt 223.98

And also in the clustering
Cluster 1326 283.0290 224.10
[(1263, 1), (1327, 1), (1385, 1)]
- id (1263, 1) mass 238.0311 rt 223.98
- id (1327, 1) mass 284.0365 rt 223.98
- id (1385, 1) mass 256.0401 rt 223.98

But missing in the output ??!!
